In [3]:
import os
import sys
import json
import urllib.request
from pyunpack import Archive
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re 

# Store the url as a string scalar: url => str
url = "https://github.com/sealneaward/nba-movement-data/tree/master/data"

# Issue request: r => requests.models.Response
r = requests.get(url)

# Extract text: html_doc => str
html_doc = r.text

# Parse the HTML: soup => bs4.BeautifulSoup
soup = BeautifulSoup(html_doc)

# Find all 'a' tags (which define hyperlinks): a_tags => bs4.element.ResultSet
a_tags = soup.find_all('a')

# Store a list of urls ending in .csv: urls => list
urls = ['https://raw.githubusercontent.com'+re.sub('/blob', '', link.get('href')) 
        for link in a_tags  if '.7z' in link.get('href')]

# Store a list of Data Frame names to be assigned to the list: df_list_names => list
df_list_names = [url.split('.7z')[0].split('/')[url.count('/')] for url in urls]

file_names = []
for file in df_list_names:
    if (('GSW' not in file) and ('ATL' not in file) and ('CHA' not in file) and ('MIA' not in file) and 
        ('ORL' not in file) and ('WAS' not in file) and ('CHI' not in file)
       and ('CLE' not in file) and ('DET' not in file) and ('IND' not in file) and ('MIL' not in file)
       and ('BOS' not in file) and ('BKN' not in file) and ('TOR' not in file) and ('NYK' not in file) and
       ('PHI' not in file) and ('DEN' not in file) and ('MIN' not in file) and ('OKC' not in file) and 
       ('POR' not in file) and ('UTA' not in file) and ('LAC' not in file) and ('LAL' not in file) and
       ('PHX' not in file) and ('SAC' not in file) and ('DAL' not in file) and ('HOU' not in file) and 
        ('MEM' not in file) and ('NOP' in file)):
        file_names.append(file)

In [4]:
# len(file_names)
# file_names

['11.20.2015.SAS.at.NOP']

In [1]:
count = 0
movement_headers = ["team_id", "player_id", "x_loc", "y_loc", "radius", "game_clock", "shot_clock", "quarter", "game_id",
                    "event_id"]


for x in file_names:
    urllib.request.urlretrieve("https://raw.githubusercontent.com/sealneaward/nba-movement-data/master/data/"+x+".7z", x+".7z")
    Archive(x+".7z").extractall("")
    os.remove(x+".7z")
    data_path = "/Users/mattbolanos/Desktop/Analytics/Analytics/Tracking.Hot.Hand/cleaning.scripts"
    csv_path = "/Users/mattbolanos/Desktop/Analytics/Analytics/Tracking.Hot.Hand/cleaning.scripts"
    files = os.listdir(data_path)
    for file in files:
        if '.json' not in file:
            continue
        try:
            count = count + 1
            file_data = open('%s/%s' % (data_path, file))
            game_id = file.replace('.json', '')
            data = json.load(file_data)
            events = data['events']
            moments = []

            for event in events:
                event_id = event['eventId']
                movement_data = event['moments']
                for moment in movement_data:
                    for player in moment[5]:
                        player.extend((moment[2], moment[3], moment[0], game_id, event_id))
                        moments.append(player)

                # movement frame is complete for game
            movement = pd.DataFrame(moments, columns=movement_headers)
            movement.to_csv('%s/%s.csv' % (csv_path, "game"+ game_id), index=False)
            os.remove(file)
            # movement.to_json('./data/json/' + game_id + '.json', orient='records')

        except Exception as e:
            print("")
            